In [10]:
# Importing all modules
import sqlite3
from tkinter import *
import tkinter.ttk as ttk
import tkinter.messagebox as mb
import tkinter.simpledialog as sd

# Connecting to Database(lib)
connector = sqlite3.connect('lib.db')
cursor = connector.cursor()
connector.execute(
'CREATE TABLE IF NOT EXISTS Lib (Name TEXT, Id TEXT PRIMARY KEY NOT NULL, Aname TEXT, Sts TEXT, Cid TEXT)'
)

# Functions Used For Maintaining The Database
def issuer_card():
    Cid = sd.askstring('Issuer Card ID', 'What is the Issuer\'s Card ID?\t\t\t')
    if not Cid:
        mb.showerror('Issuer ID cannot be zero!', 'Can\'t keep Issuer ID empty, it must have a value')
    else:
        return Cid

def display_records():
    global connector, cursor
    global tree
    tree.delete(*tree.get_children())
    curr = connector.execute('SELECT * FROM Lib')
    data = curr.fetchall()
    for records in data:
        tree.insert('', END, values=records)
        
def clear_fields():
    global Sts, Id, Name, Aname, Cid
    Sts.set('Available')
    for i in ['Id', 'Name', 'Aname', 'Cid']:
        exec(f"{i}.set('')")
        Id_entry.config(state='normal')
    try:
        tree.selection_remove(tree.selection()[0])
    except:
        pass
    
def clear_and_display():
    clear_fields()
    display_records()

def add_record():
    global connector
    global Name, Id, Aname, Sts
    if Sts.get() == 'Issued':
        Cid.set(issuer_card())
    else:
        Cid.set('N/A')
    surety = mb.askyesno('Are you sure?','Please note that ID cannot be changed Later')
    if surety:
        try:
            connector.execute(
            'INSERT INTO Lib (Name, Id, Aname, Sts, Cid) VALUES (?, ?, ?, ?, ?)',
                (Name.get(), Id.get(), Aname.get(), Sts.get(), Cid.get()))
            connector.commit()
            clear_and_display()
            mb.showinfo('Record added', 'The new record was successfully added to your database')
        except sqlite3.IntegrityError:
            mb.showerror('Book ID already in use!',
                         'The Book ID you are trying to enter is already in the database, please alter that book\'s record or check any discrepancies on your side')

def view_record():
    global Name, Id, Sts, Aname, Cid
    global tree
    if not tree.focus():
        mb.showerror('Select a row!', 'To view a record, you must select it in the table. Please do so before continuing.')
        return
    current_item_selected = tree.focus()
    values_in_selected_item = tree.item(current_item_selected)
    selection = values_in_selected_item['values']
    Name.set(selection[0])   ;   Id.set(selection[1]) ; Sts.set(selection[3])
    Aname.set(selection[2])
    try:
        Cid.set(selection[4])
    except:
        Cid.set('')

def update_record():
    def update():
        global Sts, Name, Id, Aname, Cid
        global connector, tree
        if Sts.get() == 'Issued':
            Cid.set(issuer_card())
        else:
            Cid.set('N/A')
        cursor.execute('UPDATE Lib SET Name=?, Sts=?, Aname=?, Cid=? WHERE Id=?',
                       (Name.get(), Sts.get(), Aname.get(), Cid.get(), Id.get()))
        connector.commit()
        clear_and_display()
        edit.destroy()
        Id_entry.config(state='normal')
        clear.config(state='normal')
    view_record()
    Id_entry.config(state='disable')
    clear.config(state='disable')
    edit = Button(left_frame, text='Update Record', font=btn_font, bg="blue violet", width=20, command=update)
    edit.place(x=50, y=375)

def remove_record():
    if not tree.selection():
        mb.showerror('Error!', 'Please select an item from the database')
        return
    current_item = tree.focus()
    values = tree.item(current_item)
    selection = values["values"]
    cursor.execute('DELETE FROM Lib WHERE Id=?', (selection[1], ))
    connector.commit()
    tree.delete(current_item)
    mb.showinfo('Done', 'The record you wanted deleted was successfully deleted.')
    clear_and_display()

def delete_inventory():
    if mb.askyesno('Are you sure?', 'Are you sure you want to delete the entire inventory?\n\nThis command cannot be reversed'):
        tree.delete(*tree.get_children())
        cursor.execute('DELETE FROM Lib')
        connector.commit()
    else:
        return
    
def change_availability():
    global Cid, tree, connector
    if not tree.selection():
        mb.showerror('Error!', 'Please select a book from the database')
        return
    current_item = tree.focus()
    values = tree.item(current_item)
    Id = values['values'][1]
    Sts = values["values"][3]
    if Sts == 'Issued':
        surety = mb.askyesno('Is return confirmed?', 'Has the book been returned to you?')
        if surety:
            cursor.execute('UPDATE Lib SET Sts=?, Cid=? WHERE Id=?', ('Available', 'N/A', Id))
            connector.commit()
        else: mb.showinfo(
            'Cannot be returned', 'The book status cannot be set to Available unless it has been returned')
    else:
        cursor.execute('UPDATE Lib SET Sts=?, Cid=? where Id=?', ('Issued', issuer_card(), Id))
        connector.commit()
    clear_and_display()

# Colours
lf_bg = 'Medium orchid'  # Left Color
rtf_bg = 'SpringGreen3'   # Right Top Color
rbf_bg = 'black'           # Right Bottom Color
btn_hlb_bg = 'pale green'   # Background Buttons

# Font
lbl_font = ('Georgia', 13)
entry_font = ('Times New Roman', 12)
btn_font = ('Gill Sans MT', 13)

# Main GUI Window
root = Tk()
root.title('Lib MANAGEMENT SYSTEM')
root.geometry('1010x530')
root.resizable(1,1)
Label(root, text='Lib MANAGEMENT SYSTEM', font=("Noto Sans CJK TC", 15, 'bold'), bg="grey2", fg='White').pack(side=TOP, fill=X)

# AllVariables
Sts = StringVar()
Name = StringVar()
Id = StringVar()
Aname = StringVar()
Cid = StringVar()

# Frames
left_frame = Frame(root, bg=lf_bg)
left_frame.place(x=0, y=30, relwidth=0.3, relheight=0.96)
RT_frame = Frame(root, bg=rtf_bg)
RT_frame.place(relx=0.3, y=30, relheight=0.2, relwidth=0.7)
RB_frame = Frame(root)
RB_frame.place(relx=0.3, rely=0.24, relheight=0.785, relwidth=0.7)

# Left Frame
Label(left_frame, text='Book Name', bg=lf_bg, font=lbl_font).place(x=98, y=25)
Entry(left_frame, width=25, font=entry_font, text=Name).place(x=45, y=55)
Label(left_frame, text='Book ID', bg=lf_bg, font=lbl_font).place(x=110, y=105)
Id_entry = Entry(left_frame, width=25, font=entry_font, text=Id)
Id_entry.place(x=45, y=135)
Label(left_frame, text='Author Name', bg=lf_bg, font=lbl_font).place(x=90, y=185)
Entry(left_frame, width=25, font=entry_font, text=Aname).place(x=45, y=215)
Label(left_frame, text='Status of the Book', bg=lf_bg, font=lbl_font).place(x=75, y=265)
dd = OptionMenu(left_frame, Sts, *['Available', 'Issued'])
dd.configure(font=entry_font, width=12)
dd.place(x=75, y=300)
submit = Button(left_frame, text='Add new record', font=btn_font, bg="Dark Orchid", width=20, command=add_record)
submit.place(x=50, y=375)
clear = Button(left_frame, text='Clear fields', font=btn_font, bg="Dark Orchid", width=20, command=clear_fields)
clear.place(x=50, y=435)

# Right Top Frame
Button(RT_frame, text='Delete book record', font=btn_font, bg=btn_hlb_bg, width=17, command=remove_record).place(x=8, y=30)
Button(RT_frame, text='Delete full inventory', font=btn_font, bg=btn_hlb_bg, width=17, command=delete_inventory).place(x=178, y=30)
Button(RT_frame, text='Update book details', font=btn_font, bg=btn_hlb_bg, width=17,
       command=update_record).place(x=348, y=30)
Button(RT_frame, text='Change Book Availability', font=btn_font, bg=btn_hlb_bg, width=19,
       command=change_availability).place(x=518, y=30)

# Right Bottom Frame
Label(RB_frame, text='BOOK INVENTORY',fg="White",bg=rbf_bg, font=("Noto Sans CJK TC", 15, 'bold')).pack(side=TOP, fill=X)
tree = ttk.Treeview(RB_frame, selectmode=BROWSE, columns=('Book Name', 'Book ID', 'Author', 'Status', 'Issuer Card ID'))
XScrollbar = Scrollbar(tree, orient=HORIZONTAL, command=tree.xview)
YScrollbar = Scrollbar(tree, orient=VERTICAL, command=tree.yview)
XScrollbar.pack(side=BOTTOM, fill=X)
YScrollbar.pack(side=RIGHT, fill=Y)
tree.config(xscrollcommand=XScrollbar.set, yscrollcommand=YScrollbar.set)
tree.heading('Book Name', text='Book Name', anchor=CENTER)
tree.heading('Book ID', text='Book ID', anchor=CENTER)
tree.heading('Author', text='Author', anchor=CENTER)
tree.heading('Status', text='Status of the Book', anchor=CENTER)
tree.heading('Issuer Card ID', text='Card ID of the Issuer', anchor=CENTER)
tree.column('#0', width=0, stretch=NO)
tree.column('#1', width=225, stretch=NO)
tree.column('#2', width=70, stretch=NO)
tree.column('#3', width=150, stretch=NO)
tree.column('#4', width=105, stretch=NO)
tree.column('#5', width=132, stretch=NO)
tree.place(y=30, x=0, relheight=0.9, relwidth=1)
clear_and_display()

# Finalizing the window
root.update()
root.mainloop()